**Extracting Kitchen Reviews from S3 data source**

In [1]:
import os
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()



Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:13 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-04-24 16:51:44--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-04-24 16:51:45 (5.98 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
kitchen_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Kitchen_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

kitchen_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [5]:
#Show number of rows in the dataset
kitchen_df.count()

4880466

**Transform**

In [6]:
#Transform the dataset to fit the tables in the schema file. Be sure the DataFrames match in data type and in column name.
clean_2_df = kitchen_df.dropna()
duplicate_2_df = kitchen_df.distinct()

In [7]:
#creating the review_id_table
review_id_table_df = duplicate_2_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1002B10PD9ZFM|   24197845|B000LTOCSG|     630072149| 2009-07-20|
|R1007DQP1BG5WJ|   17990743|B007M2OHEY|     362955205| 2014-09-12|
|R100AIJ0ZFN8CZ|   33816354|B0001ZYYQU|     331822491| 2007-03-09|
|R100C48B994FWD|   13231777|B00IK59T1S|     744019201| 2014-11-26|
|R100E300CKVFYD|   53006884|B00833DQCU|     298219798| 2013-12-08|
|R100GBKBV767AA|   14608024|B0000CDVD2|     184834728| 2014-04-30|
|R100I1XICZFNHD|   46627762|B0014NKJTO|      21858491| 2014-03-04|
|R100KXBN0DJ74A|   42223395|B000P9GWFS|     826643513| 2012-07-04|
|R100VC4OCYFFWM|   29497970|B0016KXAD2|     641397529| 2015-08-26|
|R100YFR4KUB3GC|   13368891|B003LYVXJ4|     510828298| 2013-12-08|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [8]:
# ensuring that dataFrames match in data type and in column name
review_id_table_df.schema

StructType(List(StructField(review_id,StringType,true),StructField(customer_id,IntegerType,true),StructField(product_id,StringType,true),StructField(product_parent,IntegerType,true),StructField(review_date,StringType,true)))

In [9]:
from pyspark.sql.types import * 

review_id_table_df = review_id_table_df.withColumn("review_date",review_id_table_df["review_date"].cast(DateType()))

review_id_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [10]:
#Create products table
products_table_df = duplicate_2_df.select(["product_id", "product_title"]).distinct()

In [11]:
# ensuring that dataFrames match in data type and in column name
products_table_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [12]:
#create customers table and ensure that dataFrames match in data type and in column name
customers_table_df = duplicate_2_df.groupby("customer_id").count().withColumnRenamed('count', 'customer_count')

In [13]:
#ensuring that dataFrames match in data type and in column name
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
customers_table_df =  customers_table_df.select(["customer_id", F.col("customer_count").cast(IntegerType())])
customers_table_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [14]:
#create vine table and ensure that dataFrames match in data type and in column name
vine_table_df = duplicate_2_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



 **Load - DataFrames into an RDS instance**

In [15]:
mode="append"
jdbc_url = "jdbc:postgresql://<endpoint REMOVED>/kitchendb"
config = {"user":"root",
          "password": "<PASSWORD REMOVED>",
          "driver":"org.postgresql.Driver"}

In [16]:
#Write dataframe to review_id_table
review_id_table_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [17]:
#Write dataframe to products_table_df
products_table_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [18]:
#Write dataframe to customers_table_df
customers_table_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [19]:
#Write dataframe to vine_table_df
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)